In [1]:
import pandas as pd
data = pd.read_json("./datachallenge_starterpack/contents.json")
data['tags']=data.explode('contents')[1::2]['contents']
data['contents']=data.explode('contents')[::2]['contents']
data = data.drop(['n'],axis=1)

In [2]:
data['label'] = data['tags'].agg(lambda x: '__label__' + ',__label__'.join(map(str, x)))
data['label'] = data['label'].replace(' ','-',regex=True)
data['label'] = data['label'].replace(',',' ',regex=True)
data = data.drop(['tags'],axis =1)
data = data[['label','contents']]

In [3]:
data['contents']=data['contents'].replace(r'\r+|\n+|\t+',' ', regex=True)
data['label']=data['label'].apply(str.lower)
data['contents']=data['contents'].apply(str.lower)

In [4]:
import json
output_folder = "./F/"
text_users = json.load(open(output_folder + "users_and_text.json"))
comm_and_leaders = json.load(open(output_folder + "communities.json"))
leaders = comm_and_leaders["leaders"]
coms = comm_and_leaders["communities"]

In [5]:
def flatten(l):
    return [item for sublist in l for item in sublist]
a = pd.read_json(output_folder + "users_and_text.json")
a = a.T
a = a.apply(flatten)
a = a[a['text'].map(lambda d: len(d)) > 0]
a


,text,text_root
0,"[empreindre, climat, mesure]",Voilà mon empreinte climat. Mesure la tienne !...
3,"[audience, crucial, climat, conservateur, cour...",Audience cruciale pour le climat devant la trè...
5,"[journéemondialedestoilette, blaguer, journée,...",C'est la #JournéeMondialeDesToilettes ??!!? Sa...
6,"[@senek99, @AvatarNeuronal, @Topper_Fr, Rempla...",@Senek99 @AvatarNeuronal @Topper_Fr Remplacer ...
7,"[@jonathanlamoni3, @teissir, @albinwagener, ré...",@JonathanLamoni3 @teissir_ @albinwagener Même ...
...,...,...
1605833,"[ruban, rouge, criminalisation, activiste, cli...","Derrière le ruban rouge, la criminalisation de..."
1605834,"[timidité, arbre, communiquer, plante, , @fra...",Timidité des arbres : comment communiquent les...
1605835,"[beau, site, histoire, colorer, morne, environ...","Un très beau site, histoire de colorer un peu ..."
1605836,"[venir, signer, appel, ➕, place, climat, débat...",Je viens de signer l'appel pour ➕ de place pou...


In [6]:
import re
def remo(text):
    X = ["@","#"]
    res = ''.join([i for i in text if not i in X])
    return res
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    res = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', emoji_pattern.sub(r'', string))
    res = ''.join([i for i in res if not i.isdigit()])
    res = res.lower()
    res = remo(res)
    return res
def spaceremove(text):
    res = [sub.replace(' ', '-') for sub in text]
    res = [sub.replace('_', '-') for sub in res]
    return res

In [8]:
a['text']=a['text'].apply(spaceremove).agg(lambda x: ' '.join(map(str, x))).apply(remove_emoji).str.split()
a['text_root'] = a['text_root'].apply(remove_emoji)
a['label'] = a['text'].agg(lambda x: '__label__' + ' __label__'.join(map(str, x)))
a = a.drop(['text'],axis =1)
a = a[['label','text_root']]
a['text_root']=a['text_root'].replace(r'\r+|\n+|\t+',' ', regex=True)

In [9]:
!mkdir -p './Data'
import numpy as np
np.savetxt('./Data/tweets.txt', a.values, fmt = "%s")
np.savetxt('./Data/contents.txt', data.values, fmt = "%s")